In [1]:
# import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta


In [2]:
df = pd.read_csv('BA.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.index = df['Date']

df.head()
df.describe()


,Open,High,Low,Close,Adj Close,Volume
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03
mean,243.365596,245.790724,240.910509,243.459889,232.934569,4.336390e+06
std,100.400782,101.663496,99.031425,100.380078,103.286000,2.916014e+06
min,105.120003,109.839996,102.099998,108.440002,97.979713,7.889000e+05
25%,144.139999,145.259998,142.994995,144.422497,128.582894,2.837675e+06
50%,235.375000,237.099999,234.314995,235.235000,224.305252,3.627750e+06
75%,346.279991,350.385010,341.934998,346.740006,338.587814,4.858500e+06
max,446.010010,446.010010,440.190002,440.619995,432.860138,3.692260e+07


In [3]:
df = df.drop(["Date"], axis=1)

In [4]:
#5.1. Step 1: extract technical indicators

#Define Technical Indicators
#_______________________________________________________________________

#Simple Moving Average  
sma = ta.SMA(df.Close,25)
df['MA25'] = sma
df = df.fillna(0)

# RSI
df["RSI"] = ta.RSI(df.Close,14)

# Stochastic K
df["Stoch_K"], df["Stoch_D"] = ta.STOCH(df["High"], df["Low"], df["Close"], fastk_period=14, slowk_period=3, slowk_matype=0, 
                                       slowd_period=3, slowd_matype=0)

# MACD
df['MACD'], df['MACDSignal'], df['MACDHist'] = ta.MACD(df.Close, fastperiod= 12, slowperiod= 26, signalperiod= 9)

# Williams R%
df["WILLR"] = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14) 

#df["WILLR"] = (WILLR - WILLR.min())/(WILLR.max() - WILLR.min())
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR
Date,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,328.293201,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,327.615601,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,327.376400,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,326.806400,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,326.217200,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512


In [5]:
# 5.2. Step 2: trend analysis using technical indicators

#make MA25 into a series
MA25 = df.iloc[:,6]

# Up trend is 1, No trend is 0, and Down trend is -1
df['Trend'] = MA25.rolling('5d').apply(lambda x: np.sign(x[-1] - x[0]))

df.tail()
#If close > MA25 and MA25 is rising for last 5 days then 1 (Uptrend)
#If close < MA25 and MA25 is declining for last 5 days then -1 (Downtrend)
#If neither are true then 0 (Notrend)
def f(row):
    if row['Close'] > row['MA25'] and row['Trend'] == 1:
        val = 1
    elif row['Close'] < row['MA25'] and row['Trend'] == -1:
        val = -1
    else:
        val = 0
    return val
df['Trend'] = df.apply(f, axis=1)
df.tail()

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend
Date,,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,328.293201,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010,-1
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,327.615601,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934,-1
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,327.376400,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550,-1
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,326.806400,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436,-1
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,326.217200,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512,-1


In [6]:
#I'm not sure if this is right, I would appreciate it if you double checked my work here

#5.3. Step 3: trading signal generation from trend analysis

Close = df.iloc[:,3]
df["UpTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5 + .5
df["DownTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5

def g(row):
    if row['Trend'] == 1:
        val = row['UpTrend']
    elif row['Trend'] == -1:
        val = row['DownTrend']
    else:
        val = 0
    return val
df['Trade_Signal'] = df.apply(g, axis=1)
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend,UpTrend,DownTrend,Trade_Signal
Date,,,,,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,328.293201,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010,-1,NaN,NaN,NaN
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,327.615601,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934,-1,0.5,0.0,0.0
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,327.376400,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550,-1,1.0,0.5,0.5
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,326.806400,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436,-1,1.0,0.5,0.5
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,326.217200,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512,-1,0.5,0.0,0.0


In [7]:
# df.count

In [8]:
#I added .tail(n=Length - 32) to omit values where the indicators = 0 since the machine learning agorithm trains on the values 
#of the indicators and the MACD doesn't generate values until 32 periods have passed.
Length = (df.shape[0] - 32)


df2 = (df[['MA25', 'RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trade_Signal']].copy().tail(n=Length).copy())


#I get some na values without this and I'm worried that using this and
# having those values as zero will mess with the algorithms interpretation of the data
df2 = df2.fillna(0)

# df2.describe()
df2.head()

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2015-03-20,153.3348,57.493095,70.084246,69.428796,0.000000,-33.848318,0.62037
2015-03-23,153.4580,52.079233,58.511840,66.242375,1.120628,-51.290336,0.00000
2015-03-24,153.5272,48.690097,48.287101,58.961062,0.877340,-70.000042,0.00000
2015-03-25,153.4096,40.352420,26.660591,44.486511,0.403911,-98.727850,0.00000
2015-03-26,153.1864,40.222293,17.768640,30.905444,0.023602,-77.966187,0.00000


In [9]:
#5.5. Step 5: network structure creation and training using ELM

target_column = ['Trade_Signal'] 
predictors = list(set(list(df2.columns))-set(target_column))
#line 3 here fufills the data normalization of Step 5.4
df2[predictors] = (df2[predictors] - df2[predictors].min())/(df2[predictors].max() - df2[predictors].min())

#df2.describe()
#df2.info()
df2.head()
# df2

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2015-03-20,0.118995,0.500200,0.704475,0.700117,0.401863,0.661517,0.62037
2015-03-23,0.119404,0.423915,0.582883,0.665149,0.438552,0.487097,0.00000
2015-03-24,0.119634,0.376159,0.475451,0.585244,0.430587,0.300000,0.00000
2015-03-25,0.119243,0.258675,0.248220,0.426400,0.415087,0.012722,0.00000
2015-03-26,0.118502,0.256841,0.154791,0.277360,0.402636,0.220338,0.00000


In [10]:
X = df2[predictors]

y = df2[target_column]


print(X.shape) ; print(y.shape)
# print(X)

(1226, 6)
(1226, 1)


In [11]:
Xindex = np.array([[1], [2], [3], [4], [5], [6]])

Xindex = Xindex.T

print(Xindex.shape); print(Xindex)

(1, 6)
[[1 2 3 4 5 6]]


In [12]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

In [13]:
#  Functional Expansion Block

# cx = np.outer(X, Xindex).flatten(order='F')
# cx = np.sum(cx.reshape(-1, 6), axis=1)
# cx = np.tanh(cx)

#all on one line
cx = np.tanh(np.sum(np.outer(X, Xindex).flatten(order='F').reshape(-1, 6), axis=1))

print(cx.shape); print(cx)

(7356,)
[0.99584427 0.99130674 0.97958046 ... 1.         1.         1.        ]


In [14]:

cx = cx.reshape(1, len(cx) ).T
print(cx.shape); print(cx)

(7356, 1)
[[0.99584427]
 [0.99130674]
 [0.97958046]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]


In [15]:
# cx = cx.reshape(-1, 6)
# print(cx.shape); print(cx)

In [16]:
# initialize random weights

synaptic_weights = np.random.random((1, (Length * 6)))

print(synaptic_weights.shape); print(synaptic_weights)

(1, 7356)
[[0.2191238  0.09464234 0.37599367 ... 0.60392613 0.91882527 0.5395848 ]]


In [17]:
# z = np.multiply(cx, synaptic_weights)
z = np.dot(cx, synaptic_weights)
print(z); print(z.shape)

[[0.21821318 0.09424903 0.37443114 ... 0.60141638 0.91500688 0.53734243]
 [0.2172189  0.09381959 0.37272506 ... 0.59867605 0.91083769 0.53489405]
 [0.21464939 0.09270978 0.36831605 ... 0.59159424 0.90006329 0.52856672]
 ...
 [0.2191238  0.09464234 0.37599367 ... 0.60392613 0.91882527 0.5395848 ]
 [0.2191238  0.09464234 0.37599367 ... 0.60392613 0.91882527 0.5395848 ]
 [0.2191238  0.09464234 0.37599367 ... 0.60392613 0.91882527 0.5395848 ]]
(7356, 7356)


In [ ]:
#After expanding the dimensional inputs of X to the cx array and using numpy.dot to multiply cx with synaptic_weights,
# the z array doesn't match the structure of the original output y array so I can't get the difference between
# my weights and the true values. It is both too large and not of the right structure compared to y. Thoughts?

In [ ]:
#I'm working on figure 1 of the article and I have the expanded inputs CX, and the initialized weights as z
#I don't know how to read equation 2 or the Moore-Penros generalized inverse function of equation 5. If I'm reading this correctly
#(I'm not 100% sure I am), those functions are the key to using my CX expanded output values to create W.

# I'm still new to ELM so if I'm missing something please let me know. 
#any help is appreciated
#Thank you so much !!!!

In [ ]:
#this is a for loop  I found online. It is not what is described in the article

# for iteration in range(1000):
#     z = np.dot(cx, synaptic_weights)
#     sigmoid = 1/(1+np.exp(-z))
#     error = (y - sigmoid)
# print(error); print(error.shape)

In [ ]:
# sigmoidDerivative = sigmoid * (1 - sigmoid)
# print(sigmoidDerivative)

In [ ]:
# synaptic_weights += np.dot(cx, error*sigmoidDerivative)
# print(synaptic_weights)